In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.5 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from tqdm import tqdm
import cv2
import subprocess
import requests
import glob

### Combine CSV files

In [ ]:
def load_annotations(folder_path):
    all_data = []

    for filename in tqdm(os.listdir(folder_path)):
        if filename.endswith("-activespeaker.csv"):
            file_path = os.path.join(folder_path, filename)
            video_id = filename.replace("-activespeaker.csv", "")
            df = pd.read_csv(file_path, header=None, sep=',')

            df.columns = ['video_id', 'timestamp', 'x1', 'y1', 'x2', 'y2', 'label', 'entity_id']
            df['video_file'] = video_id
            all_data.append(df)

    full_df = pd.concat(all_data, ignore_index=True)
    return full_df


In [ ]:
dir = '/content/drive/MyDrive/aifarm/centerstage'
train_labels = load_annotations(f"{dir}/data/raw/label_train")
test_labels = load_annotations(f"{dir}/data/raw/label_test")

train_labels.head()

100%|██████████| 33/33 [00:13<00:00,  2.48it/s]


,video_id,timestamp,x1,y1,x2,y2,label,entity_id,video_file
0,20TAGRElvfE,1740.00,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE
1,20TAGRElvfE,1740.04,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE
2,20TAGRElvfE,1740.07,0.504016,0.180556,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE
3,20TAGRElvfE,1740.11,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE
4,20TAGRElvfE,1740.16,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE


In [ ]:
train_labels.shape

(2676314, 9)

In [ ]:
test_labels.shape

(768307, 9)

In [ ]:
train_labels['label'].value_counts()

,count
label,
NOT_SPEAKING,1969134
SPEAKING_AUDIBLE,682404
SPEAKING_NOT_AUDIBLE,24776


Since our system is vision-only (no audio), the distinction between audible and inaudible isn't something the model can directly observe. So we treat both speaking cases as speaking

In [ ]:
filtered_df = train_labels[train_labels['label'].isin(['SPEAKING_AUDIBLE', 'SPEAKING_NOT_AUDIBLE', 'NOT_SPEAKING'])]

filtered_df['binary_label'] = filtered_df['label'].apply(
    lambda x: 1 if x in ['SPEAKING_AUDIBLE', 'SPEAKING_NOT_AUDIBLE'] else 0
)

In [ ]:
# Check Distribution
print(filtered_df['binary_label'].value_counts())

binary_label
0    1969134
1     707180
Name: count, dtype: int64


In [ ]:
BASE_DIR = '/content/drive/MyDrive/aifarm/centerstage'
os.makedirs(os.path.join(BASE_DIR, "data/processed"), exist_ok=True)

filtered_df.to_csv(f"{BASE_DIR}/data/processed/train_labels_clean.csv", index=False)
test_labels.to_csv(f"{BASE_DIR}/data/processed/test_labels_clean.csv", index=False)

In [ ]:
filtered_df

,video_id,timestamp,x1,y1,x2,y2,label,entity_id,video_file,binary_label
0,20TAGRElvfE,1740.00,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE,0
1,20TAGRElvfE,1740.04,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE,0
2,20TAGRElvfE,1740.07,0.504016,0.180556,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE,0
3,20TAGRElvfE,1740.11,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE,0
4,20TAGRElvfE,1740.16,0.504016,0.183333,0.678715,0.463889,NOT_SPEAKING,20TAGRElvfE_1740_1800:1,20TAGRElvfE,0
...,...,...,...,...,...,...,...,...,...,...
2676309,x-6CtPWVi6E,958.22,0.500000,0.029167,0.705399,0.452083,NOT_SPEAKING,x-6CtPWVi6E_0900_0960:54,x-6CtPWVi6E,0
2676310,x-6CtPWVi6E,958.25,0.500000,0.031250,0.705399,0.454167,NOT_SPEAKING,x-6CtPWVi6E_0900_0960:54,x-6CtPWVi6E,0
2676311,x-6CtPWVi6E,958.29,0.500000,0.031250,0.705399,0.454167,NOT_SPEAKING,x-6CtPWVi6E_0900_0960:54,x-6CtPWVi6E,0
2676312,x-6CtPWVi6E,958.32,0.500000,0.031250,0.705399,0.454167,NOT_SPEAKING,x-6CtPWVi6E_0900_0960:54,x-6CtPWVi6E,0


### Downloading Videos

In [ ]:
BASE_DIR = '/content/drive/MyDrive/aifarm/centerstage'
filtered_df = pd.read_csv(f"{BASE_DIR}/data/processed/train_labels_clean.csv")
test_labels = pd.read_csv(f"{BASE_DIR}/data/processed/test_labels_clean.csv")

In [ ]:
# === CONFIG ===
VIDEO_LIST_FILE = os.path.join(BASE_DIR, "data/raw/ava_video_list.txt")
DOWNLOAD_DIR = os.path.join(BASE_DIR, "data/videos")
S3_URL_PREFIX = "https://s3.amazonaws.com/ava-dataset/trainval"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Load available video filenames
with open(VIDEO_LIST_FILE, "r") as f:
    available_files = set(line.strip() for line in f.readlines())

# Get unique video IDs from both training and test sets
train_video_ids = set(filtered_df['video_id'].unique())
test_video_ids = set(test_labels['video_id'].unique())

# Union of both sets
required_ids = train_video_ids.union(test_video_ids)

# Match with available video filenames
video_files_to_download = []
for file_name in available_files:
    vid = os.path.splitext(file_name)[0]
    if vid in required_ids:
        video_files_to_download.append(file_name)

print(f"Found {len(video_files_to_download)} downloadable videos out of {len(required_ids)}")

Found 153 downloadable videos out of 153


In [ ]:
# === DOWNLOAD FUNCTION ===
def download_video(filename, save_dir):
    url = f"{S3_URL_PREFIX}/{filename}"
    local_path = os.path.join(save_dir, filename)

    if os.path.exists(local_path):
        print(f"Already downloaded: {filename}")
        return

    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded: {filename}")
    except Exception as e:
        print(f"Failed to download {filename}: {e}")

# === BATCH DOWNLOAD ===
for filename in tqdm(video_files_to_download):
    download_video(filename, DOWNLOAD_DIR)

  1%|          | 1/153 [00:10<27:21, 10.80s/it]

Downloaded: fD6VkIRlIRI.mkv


  1%|▏         | 2/153 [00:16<20:18,  8.07s/it]

Downloaded: 9mLYmkonWZQ.mkv


  2%|▏         | 3/153 [01:28<1:32:23, 36.95s/it]

...

Downloaded: t1LXrJOvPDg.mkv


 99%|█████████▉| 152/153 [36:04<00:14, 14.12s/it]

Downloaded: AN07xQokfiE.mp4


100%|██████████| 153/153 [36:17<00:00, 14.23s/it]

Downloaded: tNpZtigMc4g.mkv


### Extracting Faces from Videos

In [3]:
BASE_DIR = '/content/drive/MyDrive/aifarm/centerstage'
filtered_df = pd.read_csv(f"{BASE_DIR}/data/processed/train_labels_clean.csv")
test_labels = pd.read_csv(f"{BASE_DIR}/data/processed/test_labels_clean.csv")

In [ ]:
LABEL_TO_FOLDER = {
    'SPEAKING_AUDIBLE': 'speaking',
    'SPEAKING_NOT_AUDIBLE': 'speaking',
    'NOT_SPEAKING': 'not_speaking'
}

os.makedirs(SAVE_ROOT, exist_ok=True)

def extract_faces_from_video(video_path, video_df, save_root, split):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening {video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        print(f"Invalid FPS for {video_path}, skipping.")
        return

    h, w = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    video_id = os.path.splitext(os.path.basename(video_path))[0]
    video_annots = video_df[video_df['video_id'] == video_id]

    for _, row in video_annots.iterrows():
        frame_idx = int(float(row['timestamp']) * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        success, frame = cap.read()
        if not success:
            continue

        x1 = int(float(row['x1']) * w)
        y1 = int(float(row['y1']) * h)
        x2 = int(float(row['x2']) * w)
        y2 = int(float(row['y2']) * h)

        face = frame[y1:y2, x1:x2]
        if face.size == 0:
            continue

        label = row['label']
        class_folder = LABEL_TO_FOLDER.get(label)
        if class_folder is None:
            continue

        # === Subfolder by video ID ===
        save_dir = os.path.join(save_root, split, class_folder, video_id)
        os.makedirs(save_dir, exist_ok=True)

        # Filename: frame number + entity ID
        filename = f"{frame_idx}_{row['entity_id']}.jpg"
        save_path = os.path.join(save_dir, filename)
        cv2.imwrite(save_path, face)

    cap.release()

In [ ]:
SAVE_ROOT = "/content/drive/MyDrive/aifarm/centerstage/data/faces"
VIDEO_DIR = "/content/drive/MyDrive/aifarm/centerstage/data/videos"

def find_video_path(video_id, video_dir=VIDEO_DIR):
    matches = glob.glob(os.path.join(video_dir, f"{video_id}.*"))
    return matches[0] if matches else None

def is_video_already_extracted(video_id, save_root, split):
    speaking_path = os.path.join(save_root, split, "speaking", video_id)
    nonspeaking_path = os.path.join(save_root, split, "not_speaking", video_id)

    # Check if folder exists and contains any .jpg files
    has_speaking = os.path.exists(speaking_path) and glob.glob(os.path.join(speaking_path, "*.jpg"))
    has_nonspeaking = os.path.exists(nonspeaking_path) and glob.glob(os.path.join(nonspeaking_path, "*.jpg"))

    return has_speaking or has_nonspeaking

# Train set
for vid in tqdm(filtered_df['video_id'].unique(), desc="Train Set"):
    if is_video_already_extracted(vid, SAVE_ROOT, split="train"):
        print(f"Video {vid} already processed. Skipping.")
        continue

    path = find_video_path(vid)
    if path:
        print(f" Extracting faces from video {vid}")
        extract_faces_from_video(path, filtered_df, SAVE_ROOT, split="train")

# Test set
for vid in tqdm(test_labels['video_id'].unique(), desc="Test Set"):
    if is_video_already_extracted(vid, SAVE_ROOT, split="test"):
        print(f"Video {vid} already processed. Skipping.")
        continue

    path = find_video_path(vid)
    if path:
        print(f"\nExtracting faces from video {vid}")
        extract_faces_from_video(path, test_labels, SAVE_ROOT, split="test")

Train Set:   1%|          | 1/120 [00:56<1:52:19, 56.63s/it]

Video 20TAGRElvfE already processed. Skipping.


Train Set:   2%|▏         | 2/120 [01:42<1:38:29, 50.08s/it]

Video 4gVsDd8PV9U already processed. Skipping.


Train Set:   2%|▎         | 3/120 [02:16<1:23:41, 42.92s/it]

...

 Extracting faces from video AN07xQokfiE


Train Set:  24%|██▍       | 29/120 [6:43:28<55:47:43, 2207.29s/it]

 Extracting faces from video _a9SWtcaNj8


Train Set:  25%|██▌       | 30/120 [7:12:41<51:46:14, 2070.83s/it]

 Extracting faces from video AYebXQ8eUkM


### Analyse if the extracted data is accurate

In [7]:
def count_images_in_video_subfolder(video_id, folder_path):
    subfolder = os.path.join(folder_path, video_id)
    if not os.path.exists(subfolder):
        return 0
    pattern = os.path.join(subfolder, "*.jpg")
    return len(glob.glob(pattern))

folder = f"{BASE_DIR}/data/faces/train/speaking"  # or test/not_speaking, etc.
video_id = "2PpxiG0WU18"

count = count_images_in_video_subfolder(video_id, folder)
print(f"Video '{video_id}' has {count} image(s) in '{folder}'")

Video '2PpxiG0WU18' has 3538 image(s) in '/content/drive/MyDrive/aifarm/centerstage/data/faces/train/speaking'


In [9]:
folder = f"{BASE_DIR}/data/faces/train/not_speaking"  # or test/not_speaking, etc.
video_id = "2PpxiG0WU18"

count = count_images_in_video_subfolder(video_id, folder)
print(f"Video '{video_id}' has {count} image(s) in '{folder}'")

Video '2PpxiG0WU18' has 18023 image(s) in '/content/drive/MyDrive/aifarm/centerstage/data/faces/train/not_speaking'


In [8]:
filtered_df[(filtered_df['binary_label'] == 1) & (filtered_df['video_id'] == '2PpxiG0WU18')].shape

(3710, 10)

In [10]:
filtered_df[(filtered_df['binary_label'] == 0) & (filtered_df['video_id'] == '2PpxiG0WU18')].shape

(19148, 10)

In [ ]:
import cv2

cap = cv2.VideoCapture(f"{BASE_DIR}/data/videos/7nHkh4sP5Ks.mkv")
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:", fps)
cap.release()